In [183]:
from method import *
from public import *
from WindPy import w
import math


In [184]:

# 处理data
unit_value = Method.mergeSameSheet(file_name="固收净值", sheet_name="单位净值", path="Input_Data", header=[0,1], index_col=[0])
accu_value = Method.mergeSameSheet(file_name="固收净值", sheet_name="累计净值", path="Input_Data", header=[0,1], index_col=[0])

history_unit_value = pd.read_excel(Public.readFile("单位净值历史数据.xlsx"))
history_accu_value = pd.read_excel(Public.readFile("单位净值历史数据.xlsx"))
code_file = pd.read_excel(Public.readFile("统一三处代码.xlsx"))

code_SZ = code_file[["金富1号"]].T # 金富1号代码
unit_value.columns=code_SZ.values[0]
accu_value.columns=code_SZ.values[0]


start_date = unit_value.index[0]
end_date = unit_value.index[-1]
try:
    w.start("username=W1457909349;password=glh346glh;sitename=NJDX", waitTime=120)
    index_data = w.wsd("000906.SH", "close", start_date, end_date)
    w.close()
except:
    print("Wind Connect Error")


date_pd = []
for i in index_data.Times:
    date_pd.append(pd.Timestamp(i))
index_value = pd.DataFrame(index_data.Data[0], columns=["中证800"], index=date_pd)

Current Files: ['20221216-固收净值.xlsx', '20221215-固收净值.xlsx']
Current Files: ['20221216-固收净值.xlsx', '20221215-固收净值.xlsx']
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [185]:

unit_value=pd.concat([unit_value, index_value], ignore_index = False)
unit_value = unit_value.groupby(unit_value.index).first() 

accu_value=pd.concat([accu_value, index_value], ignore_index = False)
accu_value = accu_value.groupby(accu_value.index).first() 


In [186]:

unit_value.insert(0, "日期", unit_value.index)
# unit_value.insert(0, "Unnamed: 0", unit_value.index)
accu_value.insert(0, "日期", accu_value.index)
# accu_value.insert(0, "Unnamed: 0", accu_value.index)
unit_value = unit_value.fillna(0)
accu_value = accu_value.fillna(0)
unit_value = unit_value[unit_value["中证800"] != 0]
accu_value = accu_value[accu_value["中证800"] != 0]

history_unit_value = pd.concat([history_unit_value, unit_value], ignore_index = True)
history_accu_value = pd.concat([history_accu_value, accu_value], ignore_index = True)
history_unit_value = history_unit_value.fillna(0)
history_accu_value = history_accu_value.fillna(0)


history_accu_value=Method.fillWithLastDay(history_accu_value, start_col=2, start_row=2)
history_unit_value=Method.fillWithLastDay(history_unit_value, start_col=2, start_row=2)
# return history_unit_value, history_accu_value
history_accu_value.to_csv("Output_Data/mid_1.csv")
history_unit_value.to_csv("Output_Data/mid_2.csv")


Start Position Col:  2 Row:  2 DataFrame Shape: (748, 156)
Start Position Col:  2 Row:  2 DataFrame Shape: (748, 156)


In [187]:
df_单位净值=pd.read_csv(Public.readFile("mid_1.csv"), index_col=0)
lst_最新一天单位净值 = []
lst_最新一天的日期 = []
for j in range(2,df_单位净值.shape[1]):
    idx = np.where(df_单位净值.iloc[:,j] !=0.0)[0][-1]
    lst_最新一天单位净值 = lst_最新一天单位净值 + [df_单位净值.iloc[idx,j]]
    lst_最新一天的日期 = lst_最新一天的日期 + [df_单位净值.index[idx]]
    
df_最新一天单位净值 = pd.DataFrame(df_单位净值.iloc[-1,2:].T)  #先粗略地用最后一行的代表最新的单位净值
df_最新一天单位净值["产品代码"] = df_最新一天单位净值.index
df_最新一天单位净值.columns = ["最新一天单位净值", "产品代码"]
df_最新一天单位净值["最新一天单位净值"] = lst_最新一天单位净值  #再替换成真正的最新一天的单位净值
df_最新一天单位净值["最新一天的日期"] = lst_最新一天的日期
df_最新一天单位净值["最新一天的日期"] = [str(i).split(" ")[0] for i in df_最新一天单位净值["最新一天的日期"]]


#开始填充底层资产sample
df_底层资产sample = pd.read_excel(Public.readFile("持仓日报表模板.xlsx"),sheet_name="底层资产模板")
df_底层资产sample.index = df_底层资产sample["产品代码"]
for i in range(0,df_底层资产sample.shape[0]):
    产品代码 = df_底层资产sample.iloc[i,1]
    df_底层资产sample.iloc[i,10] = float(df_最新一天单位净值.loc[df_最新一天单位净值["产品代码"] == 产品代码]["最新一天单位净值"]) 
#double
df_底层资产sample["成本价（元/份）"] = df_底层资产sample["投资金额（万元）"]/df_底层资产sample["持有份额（份）"]*10000
df_底层资产sample["持仓市值（万元）"] = df_底层资产sample["持有份额（份）"] * df_底层资产sample["单位净值（元/份）"]/10000

持仓总市值 = df_底层资产sample["持仓市值（万元）"].sum()
df_底层资产sample["持仓市值占总持仓比例"] = df_底层资产sample["持仓市值（万元）"] /持仓总市值
f1 = lambda x :'%.2f%%'  %  (x*100)
df_底层资产sample[["持仓市值占总持仓比例"]] = df_底层资产sample[["持仓市值占总持仓比例"]].applymap(f1)
df_底层资产sample["2022年浮动盈亏（万元）"]= (df_底层资产sample["单位净值（元/份）"] - df_底层资产sample["2021年12月31日单位净值"])*df_底层资产sample["持有份额（份）"]/10000

df_底层资产sample["已实现盈亏（万元）"].fillna(0,inplace=True)
df_底层资产sample["2022年盈亏比例"] = (df_底层资产sample["2022年浮动盈亏（万元）"]+df_底层资产sample["已实现盈亏（万元）"])/df_底层资产sample["投资金额（万元）"]
df_底层资产sample[["2022年盈亏比例"]] = df_底层资产sample[["2022年盈亏比例"]].applymap(f1)
df_底层资产sample["累计浮动盈亏（万元）"] = (df_底层资产sample["单位净值（元/份）"] - df_底层资产sample["成本价（元/份）"])*df_底层资产sample["持有份额（份）"]/10000

df_底层资产sample["累计盈亏比例"] = (df_底层资产sample["累计浮动盈亏（万元）"]+df_底层资产sample["已实现盈亏（万元）"])/df_底层资产sample["投资金额（万元）"]
df_底层资产sample[["累计盈亏比例"]] = df_底层资产sample[["累计盈亏比例"]].applymap(f1)


series_各管理人持仓 = df_底层资产sample.groupby(by=["产品管理人"])["投资金额（万元）"].sum()
df_各管理人持仓 = series_各管理人持仓.reset_index()
for i in range(0,df_底层资产sample.shape[0]):
    产品管理人 = df_底层资产sample.iloc[i,3]
    df_底层资产sample.iat[i,-5]= df_各管理人持仓.loc[df_各管理人持仓["产品管理人"] == 产品管理人]["投资金额（万元）"]

for i in range(0,df_底层资产sample.shape[0]):
    if math.isnan(df_底层资产sample["产品规模（总份额）"].iloc[i]):
        df_底层资产sample.iloc[i,-3]=df_底层资产sample["产品规模（总份额）"].iloc[i]
    else:
        df_底层资产sample.iloc[i,-3] = format((df_底层资产sample["持有份额（份）"].iloc[i] / df_底层资产sample["产品规模（总份额）"].iloc[i]),".2%")
        
#填充号的底层资产sample就是底层资产私募配置情况表了
df_底层资产私募配置情况 = df_底层资产sample
df_hh = df_底层资产私募配置情况.fillna(0,inplace=False)
idx = []
for i in range(0,64):
    if df_hh.iloc[i,0] != 0:
        idx = idx + [i]
    else:
        df_hh.iloc[i,0] = df_hh.iloc[i,0]
        
for i in range(0,idx[1]):
    df_hh.iloc[i,0] = "一对一产品"
for i in range(idx[1],df_hh.shape[0]):
    df_hh.iloc[i,0] = "一对多产品"

df_底层资产私募配置情况.iloc[:,0] = df_hh.iloc[:,0]

In [188]:
history_accu_value=pd.read_csv("Output_Data/mid_1.csv", index_col=0)
history_unit_value=pd.read_csv("Output_Data/mid_2.csv", index_col=0)


In [189]:
股票多头_index = np.where((history_accu_value.iloc[[1]] == "股票多头") == True)[1]
指数增强_index = np.where((history_accu_value.iloc[[1]] == "指数增强") == True)[1]
空气指增_index = np.where((history_accu_value.iloc[[1]] == "空气指增") == True)[1]
量化择时_index = np.where((history_accu_value.iloc[[1]] == "量化择时") == True)[1]
量化对冲_index = np.where((history_accu_value.iloc[[1]] == "量化对冲") == True)[1]
多策略灵活配置_index = np.where((history_accu_value.iloc[[1]] == "多策略灵活配置") == True)[1]
宏观对冲_index = np.where((history_accu_value.iloc[[1]] == "宏观对冲") == True)[1]
量化期货_index = np.where((history_accu_value.iloc[[1]] == "量化期货") == True)[1]

In [190]:
股票多头_unit_value = history_unit_value.iloc[:,股票多头_index]
股票多头_unit_value = history_unit_value[["中证800"]].join(股票多头_unit_value)
股票多头_unit_value = history_unit_value[["日期"]].join(股票多头_unit_value)

指数增强_unit_value = history_unit_value.iloc[:,指数增强_index]
指数增强_unit_value = history_unit_value[["中证800"]].join(指数增强_unit_value)
指数增强_unit_value = history_unit_value[["日期"]].join(指数增强_unit_value)

空气指增_unit_value = history_unit_value.iloc[:,空气指增_index]
空气指增_unit_value = history_unit_value[["中证800"]].join(空气指增_unit_value)
空气指增_unit_value = history_unit_value[["日期"]].join(空气指增_unit_value)

量化择时_unit_value = history_unit_value.iloc[:,量化择时_index]
量化择时_unit_value = history_unit_value[["中证800"]].join(量化择时_unit_value)
量化择时_unit_value = history_unit_value[["日期"]].join(量化择时_unit_value)

量化对冲_unit_value = history_unit_value.iloc[:,量化对冲_index]
量化对冲_unit_value = history_unit_value[["中证800"]].join(量化对冲_unit_value)
量化对冲_unit_value = history_unit_value[["日期"]].join(量化对冲_unit_value)

多策略灵活配置_unit_value = history_unit_value.iloc[:,多策略灵活配置_index]
多策略灵活配置_unit_value = history_unit_value[["中证800"]].join(多策略灵活配置_unit_value)
多策略灵活配置_unit_value = history_unit_value[["日期"]].join(多策略灵活配置_unit_value)

宏观对冲_unit_value = history_unit_value.iloc[:,宏观对冲_index]
宏观对冲_unit_value = history_unit_value[["中证800"]].join(宏观对冲_unit_value)
宏观对冲_unit_value = history_unit_value[["日期"]].join(宏观对冲_unit_value)

量化期货_unit_value = history_unit_value.iloc[:,量化期货_index]
量化期货_unit_value = history_unit_value[["中证800"]].join(量化期货_unit_value)
量化期货_unit_value = history_unit_value[["日期"]].join(量化期货_unit_value)


In [200]:
INDEX_RATE = {}
INDEX_RATE["中证500"] = 0.12
INDEX_MAXDRAWDOWN={}
INDEX_MAXDRAWDOWN["中证500"] = 0.15

In [237]:

def thresholdCal(dataFrame, threshold_desc, output_threshold, df_底层资产私募配置情况) -> pd.DataFrame:
    """计算每类产品的指数, 返回DataFrame"""
    result_df = pd.DataFrame(threshold_desc)

    date = dataFrame.iloc[:,[0]]
    start_date = date.iloc[2]
    end_data = date.iloc[-1]
    index = dataFrame.iloc[:,[1]]
    accu_index_rate = float(index.iloc[2]) / float(index.iloc[-1]) - 1

    for i in range(2, dataFrame.shape[1]):
        product = dataFrame.iloc[:,[i]]
        product_name = product.iloc[0]
        product_value = product.iloc[2:dataFrame.shape[0]]
        # 对每个产品计算

        # 产品累计盈亏指标
        accu_profit_rate = float(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["产品代码"] == dataFrame.columns[i]]["累计盈亏比例"].values[0][:-1]) / 100 #累计亏损指标
        # 指数最新回撤（第一天到最后一天）
        index_rate = float(INDEX_RATE["中证500"])
        # 指数最大回撤（max那天到最后一天）
        index_maxDrawdown = float(INDEX_MAXDRAWDOWN["中证500"])
        # 指数对比相对亏损
        compare_profit_rate = accu_profit_rate - index_rate 
        # 产品最新最大回撤
        try:
            new_maxDrawdown = (float(max(product_value.values)) - float(product_value.values[-1])) / float(max(product_value.values))
        except:
            print(product_name.values, "Error")
            new_maxDrawdown = 0
        # 产品历史最大回撤
        tem = []
        for value in product_value.values:
            t = float(value)
            if t == 0: continue
            tem.append(t)
        # history_maxDrawdown = Compute.maxDrawdown(tem)[0]

        # 产品持有期间盈亏
        try:
            accu_hold_profit_rate = tem[0] / tem[-1] - 1
        except:
            accu_hold_profit_rate = 0
        # 产品对比相对回撤
        compare_maxDrawdown_rate = new_maxDrawdown - index_maxDrawdown


        # 写入DataFrame
        result_df.insert(i, product_name.to_string(), 0)
        # print(accu_profit_rate, index_rate, compare_profit_rate, new_maxDrawdown, )
        if ("产品累计亏损" in output_threshold):
            result_df.iloc[output_threshold.index("产品累计亏损"), i] = accu_profit_rate

        if ("产品相对亏损_不比较" in output_threshold):
            result_df.iloc[output_threshold.index("产品相对亏损_不比较"), i] = compare_profit_rate
        elif ("产品相对亏损" in output_threshold):
            if index_rate < 0.1:
                result_df.iloc[output_threshold.index("产品相对亏损"), i] = compare_profit_rate
            else:
                result_df.iloc[output_threshold.index("产品相对亏损")+1, i] = compare_profit_rate

        if ("产品最新回撤" in output_threshold):
            result_df.iloc[output_threshold.index("产品最新回撤"), i] = new_maxDrawdown
        # if ("产品最大回撤" in output_threshold):
        #     result_df.iloc[output_threshold.index("产品最大回撤" ), i] = history_maxDrawdown
        if ("产品相对回撤" in output_threshold):
            result_df.iloc[output_threshold.index("产品相对回撤"), i] = compare_maxDrawdown_rate
        if ("持有期间跌幅" in output_threshold):
            result_df.iloc[output_threshold.index("持有期间跌幅"), i] = accu_hold_profit_rate

        if ("日期" in output_threshold):
            result_df.iloc[output_threshold.index("日期"), i] = date.iloc[-1]

    return result_df




In [238]:
# 汇总表

股票多头_threshold_desc = {
    "指标":["累计亏损指标","相对亏损指标(中证800指数累计跌幅<10%)", "相对亏损指标(中证800指数累计跌幅>=10%)","累计最大回撤","日期"],
    "阈值":["累计亏损≤18%","累计跌幅≤10%","累计跌幅≤5%","累计跌幅≤20%"," "]}
股票多头_output_threshold = ["产品累计亏损", "产品相对亏损", "", "产品最新回撤", "日期"]
股票多头_unit_value_result = thresholdCal(股票多头_unit_value, 股票多头_threshold_desc, 股票多头_output_threshold, df_底层资产私募配置情况)

指数增强_threshold_desc = {
    "指标":["累计亏损指标","相对亏损指标", "超额最大回撤","日期"],
    "阈值":["累计亏损≤18%","超额跌幅≤5%","超额回撤≤10%"," "]}
指数增强_output_threshold = ["产品累计亏损", "产品相对亏损_不比较","产品相对回撤", "日期"]
指数增强_unit_value_result = thresholdCal(指数增强_unit_value, 指数增强_threshold_desc, 指数增强_output_threshold, df_底层资产私募配置情况)


空气指增_threshold_desc = {
    "指标":["累计亏损指标","相对亏损指标","超额最大回撤","日期"],
    "阈值":["累计亏损≤18%","超额跌幅≤5%","超额回撤≤10%"," "]}
空气指增_output_threshold = ["产品累计亏损", "产品相对亏损_不比较", "产品最新回撤", "日期"]
空气指增_unit_value_result = thresholdCal(空气指增_unit_value, 空气指增_threshold_desc, 空气指增_output_threshold, df_底层资产私募配置情况)


量化择时_threshold_desc = {
    "指标":["累计亏损指标","相对亏损指标(中证800指数累计跌幅<10%)","相对亏损指标(中证800指数累计跌幅>=10%)","累计最大回撤","日期"],
    "阈值":["累计亏损≤18%","累计跌幅≤10%","累计跌幅≤5%","累计跌幅≤20%"," "]}
量化择时_output_threshold = ["产品累计亏损", "产品相对亏损", "", "产品最新回撤", "日期"]
量化择时_unit_value_result = thresholdCal(量化择时_unit_value, 量化择时_threshold_desc, 量化择时_output_threshold, df_底层资产私募配置情况)


量化对冲_threshold_desc = {
    "指标":["累计亏损指标(连续三个交易日)","累计最大回撤(连续三个交易日)","产品持有期间跌幅(连续五个交易日)","日期"],
    "阈值":["累计亏损≤8%","累计跌幅≤10%","累计跌幅≤6%"," "]}
量化对冲_output_threshold = ["产品累计亏损", "产品相对亏损_不比较", "产品最新回撤", "日期"]
量化对冲_unit_value_result = thresholdCal(量化对冲_unit_value, 量化对冲_threshold_desc, 量化对冲_output_threshold, df_底层资产私募配置情况)

多策略灵活配置_threshold_desc = {
    "指标":["累计亏损指标","累计最大回撤","日期"],
    "阈值":["累计亏损≤15%","累计跌幅≤20%"," "]}
多策略灵活配置_output_threshold = ["产品累计亏损", "产品最新回撤", "日期"]
多策略灵活配置_unit_value_result = thresholdCal(多策略灵活配置_unit_value, 多策略灵活配置_threshold_desc, 多策略灵活配置_output_threshold, df_底层资产私募配置情况)


宏观对冲_threshold_desc = {
    "指标":["累计亏损指标(连续三个交易日)","累计最大回撤(连续三个交易日)","产品持有期间跌幅(连续三个交易日)","日期"],
    "阈值":["累计亏损≤15%","累计跌幅≤20%","累计跌幅≤15%"," "]}
宏观对冲_output_threshold = ["产品累计亏损", "产品最新回撤", "日期"]
宏观对冲_unit_value_result = thresholdCal(宏观对冲_unit_value, 宏观对冲_threshold_desc, 宏观对冲_output_threshold, df_底层资产私募配置情况)


量化期货_threshold_desc = {
    "指标":["累计亏损指标(连续三个交易日)","累计最大回撤(连续三个交易日)","产品持有期间跌幅(连续三个交易日)","日期"],
     "阈值":["累计亏损≤12%","累计跌幅≤20%","累计跌幅≤10%"," "]}
量化期货_output_threshold = ["产品累计亏损", "产品最新回撤", "持有期间跌幅","日期"]
量化期货_unit_value_result = thresholdCal(量化期货_unit_value, 量化期货_threshold_desc, 量化期货_output_threshold, df_底层资产私募配置情况)



['前海锐意锐金1期'] Error
['简雍指数增强一号'] Error
['杉树瑞升7号'] Error


In [242]:
writer = pd.ExcelWriter("Output_Data/mid_5.xlsx")


df_业务资产及盈亏情况.to_excel(writer,startcol = 1,startrow=1,index=False,sheet_name="私募种子基金持仓日报表")
df_策略分布.to_excel(writer,startcol = 6,startrow=1,index=False,sheet_name="私募种子基金持仓日报表")
df_金富1号.to_excel(writer, startcol = 13, startrow = 3,index=False,sheet_name="私募种子基金持仓日报表")
df_底层资产私募配置情况.to_excel(writer,startcol = 1, startrow = 11,index=False,sheet_name="私募种子基金持仓日报表")

股票多头_unit_value_result.to_excel(writer, startcol=0, startrow=0,index=False,sheet_name="股票多头策略")
指数增强_unit_value_result.to_excel(writer, startcol=0, startrow=0,index=False,sheet_name="指数增强策略")
空气指增_unit_value_result.to_excel(writer, startcol=0, startrow=0,index=False,sheet_name="空气指增策略")
量化择时_unit_value_result.to_excel(writer, startcol=0, startrow=0,index=False,sheet_name="量化择时策略")
量化对冲_unit_value_result.to_excel(writer, startcol=0, startrow=0, index=False, sheet_name="量化对冲策略")
多策略灵活配置_unit_value_result.to_excel(writer, startcol=0, startrow=0,index=False,sheet_name="多策略灵活配置策略")
宏观对冲_unit_value_result.to_excel(writer, startcol=0, startrow=0, index=False, sheet_name="宏观对冲")
量化期货_unit_value_result.to_excel(writer, startcol=0, startrow=0, index=False, sheet_name="量化期货")


history_unit_value.to_excel(writer, startcol=0, startrow=0, index=False, sheet_name="单位净值")
history_accu_value.to_excel(writer, startcol=0, startrow=0, index=False, sheet_name="累计净值")

writer.save()